In [18]:
print("test")

test


# 라이브러리

In [19]:
# 한국어 텍스트 감정 분석을 위한 필수 라이브러리들
from collections import Counter  # 카운터 자료구조
import os  # 운영체제 인터페이스
import platform  # 플랫폼 정보
import re  # 정규 표현식
import sys  # 시스템 정보
import warnings  # 경고 메시지 제어

import matplotlib.pyplot as plt  # 데이터 시각화
plt.rc("font", family="NanumBarunGothic")  # 한글 폰트 설정(없으면 설치 필요)

import numpy as np  # 수치 연산
import pandas as pd  # 데이터 처리 및 분석
import seaborn as sns  # 고급 시각화
import torch  # 딥러닝 프레임워크
import koreanize_matplotlib
import optuna  # 하이퍼파라미터 최적화

# 머신러닝 관련 라이브러리
from sklearn.metrics import accuracy_score, f1_score  # 평가 지표
from sklearn.model_selection import train_test_split  # 데이터 분할

# 트랜스포머 및 BERT 관련 라이브러리
from transformers import (
    AutoModelForSequenceClassification,  # 시퀀스 분류 모델
    AutoTokenizer,  # 토크나이저
    DataCollatorWithPadding,  # 패딩 데이터 콜레이터
    set_seed,
    Trainer,  # 트레이너
    TrainingArguments,  # 훈련 설정
)

# PyTorch 데이터 처리
from torch.utils.data import Dataset  # 데이터셋 및 데이터로더
# K-Fold 교차 검증
from sklearn.model_selection import StratifiedKFold

In [20]:
# GPU 메모리 정리
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# GPU 사용여부 확인

In [21]:
# 경고 메시지 필터링
warnings.filterwarnings("ignore")

# GPU 사용 가능 여부 확인
print("\n=== PyTorch GPU 지원 정보 ===")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 버전: {torch.version.cuda}")
    print(f"GPU 개수: {torch.cuda.device_count()}")
    print(f"현재 GPU: {torch.cuda.current_device()}")
    print(f"GPU 이름: {torch.cuda.get_device_name()}")
else:
    print("CPU에서 실행 중")


=== PyTorch GPU 지원 정보 ===
CUDA 사용 가능: True
CUDA 버전: 12.4
GPU 개수: 1
현재 GPU: 0
GPU 이름: Tesla V100-SXM2-32GB


# 랜덤 시드 설정

In [22]:
# 랜덤 시드 설정
RANDOM_STATE = 42
set_seed(RANDOM_STATE)

print(f"랜덤 시드 {RANDOM_STATE}로 설정 완료")

랜덤 시드 42로 설정 완료


# 데이터 불러오기

In [23]:
# 데이터 로드
df = pd.read_csv("../data/train.csv")

# 전처리 파이프라인 정의

In [24]:
# 텍스트 전처리 파이프라인 클래스 구성
class TextPreprocessingPipeline:
    
    def __init__(self):
        self.is_fitted = False
        self.vocab_info = {}
        self.label_patterns = {}

    def basic_preprocess(self, texts):
        processed_texts = []
        for text in texts:
            # 기본 텍스트 정리
            cleaned = self._clean_text(text)
            # 정규화
            normalized = self._normalize_text(cleaned)
            processed_texts.append(normalized)
        return processed_texts

    def _clean_text(self, text):
        if pd.isna(text):
            return ""

        text = str(text).strip()
        text = re.sub(r"[ㄱ-ㅊㅌㅍㅏ-ㅛㅡ-ㅣ]+", "", text)
        text = re.sub(r"([ㅋㅎ])\1{2,}", r"\1\1", text)
        text = re.sub(r"([ㅠㅜㅡ])\1{2,}", r"\1\1", text)
        text = re.sub(r"(.)\1{3,}", r"\1\1\1", text)
        text = re.sub(r"[^\w\s가-힣.,!?ㅋㅎㅠㅜㅡ~\-]", " ", text)
        text = re.sub(r"\s+", " ", text)

        return text.strip()

    def _normalize_text(self, text):
        # 소문자 변환
        text = text.lower()

        # 구두점 정규화
        text = re.sub(r"[.]{2,}", ".", text)
        text = re.sub(r"[!]{2,}", "!", text)
        text = re.sub(r"[?]{2,}", "?", text)
        text = re.sub(r"[,]{2,}", ",", text)
        text = re.sub(r"\s+([.,!?])", r"\1", text)
        text = re.sub(r"([.,!?])\s+", r"\1 ", text)

        # 특수문자 정리
        text = re.sub(
            r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+",
            "",
            text,
        )
        text = re.sub(r"\S+@\S+", "", text)
        text = re.sub(r"@\w+", "", text)
        text = re.sub(r"\s+", " ", text)

        return text.strip()

    def fit(self, texts, labels=None):
        self.is_fitted = True
        # print("✓ 전처리 파이프라인 완료")

    def transform(self, texts):
        if not self.is_fitted:
            print("Warning: 파이프라인이 학습되지 않았습니다. 기본 전처리만 적용합니다.")
        return self.basic_preprocess(texts)

    def fit_transform(self, texts, labels=None):
        self.fit(texts, labels)
        return self.transform(texts)

# 전처리 파이프라인 생성

In [25]:
# 전처리 파이프라인 인스턴스 생성
preprocessor = TextPreprocessingPipeline()
print("텍스트 전처리 파이프라인 클래스 구성 완료")
print("현재: 기본 전처리 기능 구현됨")

텍스트 전처리 파이프라인 클래스 구성 완료
현재: 기본 전처리 기능 구현됨


# 환경 변수 설정

In [26]:
# 환경변수 설정 (토크나이저 병렬처리 경고 해결)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# GPU 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "4,5"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# 토크나이저 및 모델 설정
NUM_CLASSES = 4  # 클래스 개수

# GPU 설정

In [27]:
# GPU 메모리 정리 및 확인
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU {torch.cuda.device_count()}개 사용 가능: {device}")
    for i in range(torch.cuda.device_count()):
        print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("⚠️  CUDA 사용 불가 - CPU로 훈련 진행")

✅ GPU 1개 사용 가능: cuda
   GPU 0: Tesla V100-SXM2-32GB


# [Class] ReviewDataset

In [28]:
class ReviewDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts, self.labels, self.tokenizer, self.max_length = (
            texts,
            labels,
            tokenizer,
            max_length,
        )

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # 텍스트 토크나이징 및 패딩
        encoding = self.tokenizer(
            str(self.texts.iloc[idx]),
            truncation=True,  # 최대 길이 초과시 자르기
            padding="max_length",  # 최대 길이까지 패딩
            max_length=self.max_length,
            return_tensors="pt",  # PyTorch 텐서로 반환
        )

        # 기본 아이템 구성 (input_ids, attention_mask)
        item = {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
        }

        # labels가 None이 아닌 경우에만 추가 (train/valid용)
        if self.labels is not None:
            item["labels"] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)

        return item

# [Function] compute_metrics

In [29]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # 예측값에서 가장 높은 확률의 클래스 선택
    predictions = np.argmax(predictions, axis=1)

    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),  # 클래스별 가중평균 F1
    }

# 하이퍼파라미터 정의

In [38]:
# 토크나이저 및 모델 설정
CHOSEN_MAX_LENGTH = {"../tapt-electra": 128, "../tapt-roberta": 128, "../tapt-bert": 128}
NUM_CLASSES = 4  # 클래스 개수 (0: 강한부정, 1: 약한부정, 2: 약한긍정, 3: 강한긍정)

# 배치 크기 설정
BATCH_SIZE_TRAIN = {"../tapt-electra": 128, "../tapt-roberta": 64, "../tapt-bert": 64}
BATCH_SIZE_EVAL = {"../tapt-electra": 64, "../tapt-roberta": 64, "../tapt-bert": 64}

# 훈련 하이퍼파라미터
LEARNING_RATE = {"../tapt-electra": 1.276836788492535e-05, "../tapt-roberta": 1.131155727098881e-05, "../tapt-bert": 1.422517849886189e-05}
NUM_EPOCHS = {"../tapt-electra": 3, "../tapt-roberta": 3, "../tapt-bert": 4}
WARMUP_STEPS = {"../tapt-electra": 682, "../tapt-roberta": 668, "../tapt-bert": 696}
WEIGHT_DECAY = {"../tapt-electra": 0.04553144387239281, "../tapt-roberta": 0.0980186282772229, "../tapt-bert": 0.002246181281329394}

# 모델 저장
SAVE_MODEL = False
# wandb 사용 여부
USE_WANDB = True

# 데이터 분할

In [39]:
# 전체 데이터 분할
X = df['review']
y = df['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)

X_train_processed = preprocessor.fit_transform(X_train.tolist(), y_train.tolist())
X_val_processed = preprocessor.transform(X_val.tolist())

X_train_processed = pd.Series(X_train_processed)
X_val_processed = pd.Series(X_val_processed)

# 앙상블 모델 정의

In [40]:
model_names = [
    "../tapt-electra",
    "../tapt-roberta",
    "../tapt-bert"
]

trainers = []
tokenizers = []
models = []

# [Function] 모델 학습 정의

In [ ]:
# 함수로 훈련 간소화
def train_model(model_name):
    print(f"🤖 모델 로딩 및 훈련: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_CLASSES)

    train_dataset = ReviewDataset(X_train_processed, y_train, tokenizer, CHOSEN_MAX_LENGTH[model_name])
    val_dataset = ReviewDataset(X_val_processed, y_val, tokenizer, CHOSEN_MAX_LENGTH[model_name])

    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=NUM_EPOCHS[model_name],
        per_device_train_batch_size=BATCH_SIZE_TRAIN[model_name],
        per_device_eval_batch_size=BATCH_SIZE_EVAL[model_name],
        warmup_steps=WARMUP_STEPS[model_name],
        weight_decay=WEIGHT_DECAY[model_name],
        learning_rate=LEARNING_RATE[model_name],
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch" if SAVE_MODEL else "no",
        load_best_model_at_end=SAVE_MODEL,
        metric_for_best_model="accuracy" if SAVE_MODEL else None,
        greater_is_better=True,
        save_total_limit=2 if SAVE_MODEL else 0,
        report_to="wandb" if USE_WANDB else "none",  # wandb 로깅 조건부 활성화
        run_name=f"optuna_{model_name}" if USE_WANDB else None,
        seed=RANDOM_STATE,
        fp16=torch.cuda.is_available(),
        dataloader_num_workers=2,
        remove_unused_columns=False,
        push_to_hub=False,
        gradient_accumulation_steps=1,
        logging_first_step=True,
        save_safetensors=SAVE_MODEL,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer, tokenizer, model

In [42]:
# 각 모델 훈련 루프
for model_name in model_names:
    trainer, tokenizer, model = train_model(model_name)
    trainers.append(trainer)
    tokenizers.append(tokenizer)
    models.append(model)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at ../tapt-electra and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


🤖 모델 로딩 및 훈련: ../tapt-electra


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.421800,0.420421,0.836707,0.834419
2,0.381900,0.393344,0.847935,0.845282
3,0.343400,0.394464,0.849759,0.847728


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../tapt-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


🤖 모델 로딩 및 훈련: ../tapt-roberta


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.404400,0.406215,0.841552,0.838599
2,0.355700,0.375874,0.858162,0.855716
3,0.283800,0.385691,0.858073,0.857108


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../tapt-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


🤖 모델 로딩 및 훈련: ../tapt-bert


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.399700,0.409138,0.838924,0.835712
2,0.355200,0.379123,0.853764,0.851394
3,0.267100,0.405418,0.850009,0.849347
4,0.245300,0.418759,0.852333,0.851554


In [52]:
df = pd.read_csv("../data/train.csv")
X = df['review']
y = df['label']

In [55]:
# 함수로 훈련 간소화
def kfold_train_model(model_name):
    print(f"🤖 모델 로딩 및 훈련: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    X_processed = preprocessor.fit_transform(X.tolist(), y.tolist())
    X_processed = pd.Series(X_processed)
    
    # StratifiedKFold(K=4) 적용: 훈련 데이터에 대해 4-fold CV 수행
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=RANDOM_STATE)
    fold_metrics = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_processed, y_train)):
        print(f"🤖 Fold {fold+1}/4 시작")
        
        # 각 fold에 대한 데이터 분할 (X_train_processed와 y_train이 list 또는 array라고 가정)
        X_train_fold, X_val_fold = X_processed.iloc[train_idx], X_processed.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

        train_dataset = ReviewDataset(X_train_fold, y_train_fold, tokenizer, CHOSEN_MAX_LENGTH[model_name])
        val_dataset = ReviewDataset(X_val_fold, y_val_fold, tokenizer, CHOSEN_MAX_LENGTH[model_name])
        
        # 각 fold에 대해 새로운 모델 로딩
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_CLASSES)
        
        # TrainingArguments 설정 (fold별 output_dir 변경하여 중복 방지)
        training_args = TrainingArguments(
            output_dir=f"./results_fold_{fold}",
            num_train_epochs=1,
            per_device_train_batch_size=BATCH_SIZE_TRAIN[model_name],
            per_device_eval_batch_size=BATCH_SIZE_EVAL[model_name],
            warmup_steps=WARMUP_STEPS[model_name],
            weight_decay=WEIGHT_DECAY[model_name],
            learning_rate=LEARNING_RATE[model_name],
            logging_steps=100,
            eval_strategy="epoch",
            save_strategy="epoch" if SAVE_MODEL else "no",
            load_best_model_at_end=SAVE_MODEL,
            metric_for_best_model="accuracy" if SAVE_MODEL else None,
            greater_is_better=True,
            save_total_limit=2 if SAVE_MODEL else 0,
            report_to="none",
            seed=RANDOM_STATE,
            fp16=torch.cuda.is_available(),
            dataloader_num_workers=2,
            remove_unused_columns=False,
            push_to_hub=False,
            gradient_accumulation_steps=1,
            logging_first_step=True,
            save_safetensors=SAVE_MODEL,
        )
        
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
            compute_metrics=compute_metrics,
        )
        
        trainer.train()
        
        # fold 평가
        eval_result = trainer.evaluate()
        fold_metrics.append(eval_result)
        print(f"🤖 Fold {fold+1} 평가 결과: {eval_result}")
    
    # CV 평균 메트릭 계산 및 출력 (예: accuracy)
    if fold_metrics:
        avg_accuracy = np.mean([m['eval_accuracy'] for m in fold_metrics if 'eval_accuracy' in m])
        print(f"🤖 4-Fold CV 평균 Accuracy: {avg_accuracy:.4f}")
    
    # 최종 모델: 전체 훈련 데이터로 훈련 (원래 val 사용)
    print("🤖 최종 모델 훈련 시작 (전체 훈련 데이터 사용)")
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=NUM_CLASSES)
    
    train_dataset = ReviewDataset(X_train_processed, y_train, tokenizer, CHOSEN_MAX_LENGTH[model_name])
    val_dataset = ReviewDataset(X_val_processed, y_val, tokenizer, CHOSEN_MAX_LENGTH[model_name])
    
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,
        per_device_train_batch_size=BATCH_SIZE_TRAIN[model_name],
        per_device_eval_batch_size=BATCH_SIZE_EVAL[model_name],
        warmup_steps=WARMUP_STEPS[model_name],
        weight_decay=WEIGHT_DECAY[model_name],
        learning_rate=LEARNING_RATE[model_name],
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="epoch" if SAVE_MODEL else "no",
        load_best_model_at_end=SAVE_MODEL,
        metric_for_best_model="accuracy" if SAVE_MODEL else None,
        greater_is_better=True,
        save_total_limit=2 if SAVE_MODEL else 0,
        report_to="none",
        seed=RANDOM_STATE,
        fp16=torch.cuda.is_available(),
        dataloader_num_workers=2,
        remove_unused_columns=False,
        push_to_hub=False,
        gradient_accumulation_steps=1,
        logging_first_step=True,
        save_safetensors=SAVE_MODEL,
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )
    
    trainer.train()
    return trainer, tokenizer, model

In [56]:
model_names = [
    "../tapt-electra",
    "../tapt-roberta",
    "../tapt-bert"
]

kfold_trainers = []
kfold_tokenizers = []
kfold_models = []

for model_name in model_names:
    trainer, tokenizer, model = kfold_train_model(model_name)
    kfold_trainers.append(trainer)
    kfold_tokenizers.append(tokenizer)
    kfold_models.append(model)

🤖 모델 로딩 및 훈련: ../tapt-electra


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at ../tapt-electra and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


🤖 Fold 1/4 시작


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.455200,0.437780,0.831182,0.827849


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at ../tapt-electra and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.402700,0.406110,0.841480,0.837408


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../tapt-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


🤖 Fold 4 평가 결과: {'eval_loss': 0.4061097800731659, 'eval_accuracy': 0.8414804219560165, 'eval_f1': 0.8374077273386935, 'eval_runtime': 29.6441, 'eval_samples_per_second': 1886.719, 'eval_steps_per_second': 29.483, 'epoch': 1.0}
🤖 4-Fold CV 평균 Accuracy: 0.8411
🤖 최종 모델 훈련 시작 (전체 훈련 데이터 사용)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.398400,0.391784,0.847559,0.843614


# 모델 추론

In [57]:
# 테스트 데이터 처리 및 앙상블 추론
df_test = pd.read_csv("../data/test.csv")
test_texts = df_test["review"].tolist()
test_processed = preprocessor.transform(test_texts)
test_processed = pd.Series(test_processed)

test_datasets = []
# for i, tokenizer in enumerate(tokenizers):
#     test_datasets.append(ReviewDataset(test_processed, None, tokenizer, CHOSEN_MAX_LENGTH))
test_datasets.append(ReviewDataset(test_processed, None, tokenizers[0], CHOSEN_MAX_LENGTH["../tapt-electra"]))
test_datasets.append(ReviewDataset(test_processed, None, tokenizers[1], CHOSEN_MAX_LENGTH["../tapt-roberta"]))
test_datasets.append(ReviewDataset(test_processed, None, tokenizers[2], CHOSEN_MAX_LENGTH["../tapt-bert"]))

all_logits = []
for i, trainer in enumerate(kfold_trainers): # trainers
    pred = trainer.predict(test_datasets[i])
    all_logits.append(pred.predictions)

avg_logits = np.mean(all_logits, axis=0)
predicted_labels = np.argmax(avg_logits, axis=1)

df_test["pred"] = predicted_labels

# 결과 분석

In [58]:
# 결과 분석
LABEL_MAPPING = {0: "강한 부정", 1: "약한 부정", 2: "약한 긍정", 3: "강한 긍정"}
unique_predictions, counts = np.unique(predicted_labels, return_counts=True)
print("\n클래스별 예측 분포:")
for pred, count in zip(unique_predictions, counts):
    percentage = (count / len(predicted_labels)) * 100
    class_name = LABEL_MAPPING.get(pred, f"클래스 {pred}")
    print(f"   {class_name} ({pred}): {count:,}개 ({percentage:.1f}%)")


클래스별 예측 분포:
   강한 부정 (0): 25,975개 (43.3%)
   약한 부정 (1): 3,392개 (5.7%)
   약한 긍정 (2): 22,810개 (38.1%)
   강한 긍정 (3): 7,751개 (12.9%)


# 제출 파일 호출

In [59]:
# 제출 파일 호출
sample_submission = pd.read_csv("../data/sample_submission.csv")
submission_df = sample_submission[["ID"]].merge(df_test[["ID", "pred"]], on="ID", how="left")
submission_df = submission_df[["ID", "pred"]]

# 검증

In [60]:
assert len(submission_df) == len(sample_submission), (
    f"길이 불일치: submission_df는 {len(submission_df)}행, sample_submission은 {len(sample_submission)}행"
)
assert submission_df["pred"].isin([0, 1, 2, 3]).all(), (
    "모든 예측값은 [0, 1, 2, 3] 범위에 있어야 합니다"
)
assert not submission_df["pred"].isnull().any(), "예측값에 null 값이 있으면 안됩니다"
assert not submission_df["ID"].isnull().any(), "ID 컬럼에 null 값이 있으면 안됩니다"
print("✅ 모든 검증이 통과되었습니다!")

if torch.cuda.is_available():
    torch.cuda.empty_cache()

✅ 모든 검증이 통과되었습니다!


# 제출 파일 생성

In [61]:
# 제출 파일 저장
submission_path = "./output14.csv"
submission_df.to_csv(submission_path, index=False)
print(f"제출 파일 저장 완료: {submission_path}")

제출 파일 저장 완료: ./output14.csv


# Oputna 추가 작업

## klue/roberta-base

In [23]:
def objective_roberta(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True)
    num_epochs = trial.suggest_int("num_epochs", 2, 5)
    batch_size_train = trial.suggest_categorical("batch_size_train", [64, 128, 256])
    batch_size_eval = trial.suggest_categorical("batch_size_eval", [64, 128, 256])
    warmup_steps = trial.suggest_int("warmup_steps", 0, 1000)
    weight_decay = trial.suggest_float("weight_decay", 0.001, 0.1, log=True)
    max_length = trial.suggest_categorical("max_length", [64, 128])

    # 전체 데이터 전처리
    X_processed = preprocessor.fit_transform(X.tolist(), y.tolist())
    X_processed = pd.Series(X_processed)

    X_train, X_val, y_train, y_val = train_test_split(X_processed, y,test_size=0.2, stratify=y, random_state=RANDOM_STATE)

    train_dataset = ReviewDataset(X_train, y_train, tokenizer, max_length)
    val_dataset = ReviewDataset(X_val, y_val, tokenizer, max_length)

    model = AutoModelForSequenceClassification.from_pretrained("../tapt-roberta", num_labels=NUM_CLASSES)

    training_args = TrainingArguments(
        output_dir="./results_optuna",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size_train,
        per_device_eval_batch_size=batch_size_eval,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        learning_rate=learning_rate,
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="no",
        report_to="none",
        seed=RANDOM_STATE,
        fp16=torch.cuda.is_available(),
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

# 캐시 초기화
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Optuna 실행 (기본 모델로 최적화)
tokenizer = AutoTokenizer.from_pretrained("../tapt-roberta")  # Optuna용 토크나이저
study = optuna.create_study(direction="maximize")
study.optimize(objective_roberta, n_trials=10)
best_params_roberta = study.best_params
print(f"최적 하이퍼파라미터: {best_params_roberta}")

# 최적 파라미터 적용
roberta_BATCH_SIZE_TRAIN = best_params_roberta["batch_size_train"]
roberta_BATCH_SIZE_EVAL = best_params_roberta["batch_size_eval"]
roberta_LEARNING_RATE = best_params_roberta["learning_rate"]
roberta_NUM_EPOCHS = best_params_roberta["num_epochs"]
roberta_WARMUP_STEPS = best_params_roberta["warmup_steps"]
roberta_WEIGHT_DECAY = best_params_roberta["weight_decay"]
roberta_CHOSEN_MAX_LENGTH = best_params_roberta["max_length"]
print("roberta_BATCH_SIZE_TRAIN:", roberta_BATCH_SIZE_TRAIN)
print("roberta_BATCH_SIZE_EVAL:", roberta_BATCH_SIZE_EVAL)
print("roberta_LEARNING_RATE:", roberta_LEARNING_RATE)
print("roberta_NUM_EPOCHS:", roberta_NUM_EPOCHS)
print("roberta_WARMUP_STEPS:", roberta_WARMUP_STEPS)
print("roberta_WEIGHT_DECAY:", roberta_WEIGHT_DECAY)
print("roberta_CHOSEN_MAX_LENGTH:", roberta_CHOSEN_MAX_LENGTH)

[I 2025-10-29 18:01:47,347] A new study created in memory with name: no-name-33dd570d-bdeb-40bd-bbb4-ffd637d19cb4
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../tapt-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.434100,0.427240,0.834096,0.829281
2,0.361600,0.412934,0.842625,0.839122
3,0.244700,0.450862,0.844413,0.843758


[I 2025-10-29 18:25:08,761] Trial 0 finished with value: 0.8437576509010117 and parameters: {'learning_rate': 3.508859601126384e-05, 'num_epochs': 3, 'batch_size_train': 64, 'batch_size_eval': 64, 'warmup_steps': 770, 'weight_decay': 0.006810423821776945, 'max_length': 64}. Best is trial 0 with value: 0.8437576509010117.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../tapt-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.425200,0.443808,0.830717,0.823127
2,0.353700,0.397936,0.845754,0.842418
3,0.244100,0.419452,0.848507,0.848884
4,0.153100,0.519656,0.849383,0.849319
5,0.086700,0.609384,0.849455,0.849437


[I 2025-10-29 19:15:57,931] Trial 1 finished with value: 0.8494372629723314 and parameters: {'learning_rate': 7.960057845425662e-05, 'num_epochs': 5, 'batch_size_train': 128, 'batch_size_eval': 256, 'warmup_steps': 533, 'weight_decay': 0.07100540218860062, 'max_length': 128}. Best is trial 1 with value: 0.8494372629723314.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../tapt-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.402600,0.406502,0.839675,0.837014
2,0.354300,0.378276,0.856839,0.854554
3,0.284600,0.385231,0.857232,0.856226


[I 2025-10-29 22:41:34,769] Trial 8 finished with value: 0.8562264320036017 and parameters: {'learning_rate': 1.131155727098881e-05, 'num_epochs': 3, 'batch_size_train': 64, 'batch_size_eval': 64, 'warmup_steps': 668, 'weight_decay': 0.0980186282772229, 'max_length': 128}. Best is trial 8 with value: 0.8562264320036017.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ../tapt-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.466200,0.455486,0.822439,0.818758
2,0.406300,0.416196,0.839245,0.836437
3,0.346700,0.422374,0.841230,0.837282
4,0.290700,0.427705,0.841802,0.839925
5,0.250100,0.454421,0.842124,0.841103


[I 2025-10-29 23:12:47,407] Trial 9 finished with value: 0.8411026216920604 and parameters: {'learning_rate': 2.1804135496929557e-05, 'num_epochs': 5, 'batch_size_train': 256, 'batch_size_eval': 256, 'warmup_steps': 955, 'weight_decay': 0.06322696114512115, 'max_length': 64}. Best is trial 8 with value: 0.8562264320036017.


최적 하이퍼파라미터: {'learning_rate': 1.131155727098881e-05, 'num_epochs': 3, 'batch_size_train': 64, 'batch_size_eval': 64, 'warmup_steps': 668, 'weight_decay': 0.0980186282772229, 'max_length': 128}
roberta_BATCH_SIZE_TRAIN: 64
roberta_BATCH_SIZE_EVAL: 64
roberta_LEARNING_RATE: 1.131155727098881e-05
roberta_NUM_EPOCHS: 3
roberta_WARMUP_STEPS: 668
roberta_WEIGHT_DECAY: 0.0980186282772229
roberta_CHOSEN_MAX_LENGTH: 128


## klue/bert-base

In [24]:
def objective_bert(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True)
    num_epochs = trial.suggest_int("num_epochs", 2, 5)
    batch_size_train = trial.suggest_categorical("batch_size_train", [64, 128, 256])
    batch_size_eval = trial.suggest_categorical("batch_size_eval", [64, 128, 256])
    warmup_steps = trial.suggest_int("warmup_steps", 0, 1000)
    weight_decay = trial.suggest_float("weight_decay", 0.001, 0.1, log=True)
    max_length = trial.suggest_categorical("max_length", [64, 128])

    # 전체 데이터 전처리
    X_processed = preprocessor.fit_transform(X.tolist(), y.tolist())
    X_processed = pd.Series(X_processed)

    X_train, X_val, y_train, y_val = train_test_split(X_processed, y,test_size=0.2, stratify=y, random_state=RANDOM_STATE)

    train_dataset = ReviewDataset(X_train, y_train, tokenizer, max_length)
    val_dataset = ReviewDataset(X_val, y_val, tokenizer, max_length)

    model = AutoModelForSequenceClassification.from_pretrained("../tapt-bert", num_labels=NUM_CLASSES)

    training_args = TrainingArguments(
        output_dir="./results_optuna",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size_train,
        per_device_eval_batch_size=batch_size_eval,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        learning_rate=learning_rate,
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="no",
        report_to="none",
        seed=RANDOM_STATE,
        fp16=torch.cuda.is_available(),
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

# 캐시 초기화
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Optuna 실행 (기본 모델로 최적화)
tokenizer = AutoTokenizer.from_pretrained("../tapt-bert")  # Optuna용 토크나이저
study = optuna.create_study(direction="maximize")
study.optimize(objective_bert, n_trials=10)
best_params_bert = study.best_params
print(f"최적 하이퍼파라미터: {best_params_bert}")

# 최적 파라미터 적용
bert_BATCH_SIZE_TRAIN = best_params_bert["batch_size_train"]
bert_BATCH_SIZE_EVAL = best_params_bert["batch_size_eval"]
bert_LEARNING_RATE = best_params_bert["learning_rate"]
bert_NUM_EPOCHS = best_params_bert["num_epochs"]
bert_WARMUP_STEPS = best_params_bert["warmup_steps"]
bert_WEIGHT_DECAY = best_params_bert["weight_decay"]
bert_CHOSEN_MAX_LENGTH = best_params_bert["max_length"]
print("bert_BATCH_SIZE_TRAIN:", bert_BATCH_SIZE_TRAIN)
print("bert_BATCH_SIZE_EVAL:", bert_BATCH_SIZE_EVAL)
print("bert_LEARNING_RATE:", bert_LEARNING_RATE)
print("bert_NUM_EPOCHS:", bert_NUM_EPOCHS)
print("bert_WARMUP_STEPS:", bert_WARMUP_STEPS)
print("bert_WEIGHT_DECAY:", bert_WEIGHT_DECAY)
print("bert_CHOSEN_MAX_LENGTH:", bert_CHOSEN_MAX_LENGTH)

[I 2025-10-29 23:12:47,631] A new study created in memory with name: no-name-dac957e7-d624-475e-b1ac-e85057890883
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../tapt-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.393000,0.398963,0.843608,0.839804
2,0.332700,0.379085,0.854282,0.852303


[I 2025-10-29 23:36:03,274] Trial 0 finished with value: 0.8523030034731702 and parameters: {'learning_rate': 2.0210493315419464e-05, 'num_epochs': 2, 'batch_size_train': 64, 'batch_size_eval': 128, 'warmup_steps': 717, 'weight_decay': 0.06988079266802173, 'max_length': 128}. Best is trial 0 with value: 0.8523030034731702.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../tapt-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.442400,0.450815,0.822832,0.818615
2,0.408700,0.417973,0.836724,0.832949
3,0.366300,0.418703,0.838226,0.835276


[I 2025-10-29 23:56:41,924] Trial 1 finished with value: 0.8352760065640897 and parameters: {'learning_rate': 1.0065957980706629e-05, 'num_epochs': 3, 'batch_size_train': 128, 'batch_size_eval': 256, 'warmup_steps': 760, 'weight_decay': 0.005520088577921894, 'max_length': 64}. Best is trial 0 with value: 0.8523030034731702.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../tapt-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.441700,0.448822,0.823583,0.818588
2,0.408600,0.421942,0.836403,0.832504


[I 2025-10-30 03:05:42,371] Trial 8 finished with value: 0.8325044431286751 and parameters: {'learning_rate': 1.0751087821715655e-05, 'num_epochs': 2, 'batch_size_train': 128, 'batch_size_eval': 64, 'warmup_steps': 819, 'weight_decay': 0.002254967046687461, 'max_length': 64}. Best is trial 3 with value: 0.8524501125651771.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../tapt-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.430300,0.445580,0.826766,0.821450
2,0.341400,0.429480,0.837100,0.834439
3,0.211300,0.489499,0.834597,0.834921
4,0.118800,0.610951,0.835133,0.833540
5,0.059200,0.744766,0.835240,0.835066


[I 2025-10-30 03:39:44,412] Trial 9 finished with value: 0.8350659772882228 and parameters: {'learning_rate': 8.184238417896732e-05, 'num_epochs': 5, 'batch_size_train': 128, 'batch_size_eval': 256, 'warmup_steps': 96, 'weight_decay': 0.01807010021083869, 'max_length': 64}. Best is trial 3 with value: 0.8524501125651771.


최적 하이퍼파라미터: {'learning_rate': 1.4225178498861891e-05, 'num_epochs': 4, 'batch_size_train': 128, 'batch_size_eval': 64, 'warmup_steps': 696, 'weight_decay': 0.002246181281329394, 'max_length': 128}
bert_BATCH_SIZE_TRAIN: 128
bert_BATCH_SIZE_EVAL: 64
bert_LEARNING_RATE: 1.4225178498861891e-05
bert_NUM_EPOCHS: 4
bert_WARMUP_STEPS: 696
bert_WEIGHT_DECAY: 0.002246181281329394
bert_CHOSEN_MAX_LENGTH: 128


## monologg/koelectra-base-v3-discriminator

In [16]:
def objective_electra(trial):
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True)
    num_epochs = trial.suggest_int("num_epochs", 2, 5)
    batch_size_train = trial.suggest_categorical("batch_size_train", [64, 128, 256])
    batch_size_eval = trial.suggest_categorical("batch_size_eval", [64, 128, 256])
    warmup_steps = trial.suggest_int("warmup_steps", 0, 1000)
    weight_decay = trial.suggest_float("weight_decay", 0.001, 0.1, log=True)
    max_length = trial.suggest_categorical("max_length", [64, 128, 256])

    # 전체 데이터 전처리 (Optuna 내 반복 피함)
    X_processed = preprocessor.fit_transform(X.tolist(), y.tolist())
    X_processed = pd.Series(X_processed)

    X_train, X_val, y_train, y_val = train_test_split(X_processed, y,test_size=0.2, stratify=y, random_state=RANDOM_STATE)

    train_dataset = ReviewDataset(X_train, y_train, tokenizer, max_length)
    val_dataset = ReviewDataset(X_val, y_val, tokenizer, max_length)

    model = AutoModelForSequenceClassification.from_pretrained("../tapt-electra", num_labels=NUM_CLASSES)

    training_args = TrainingArguments(
        output_dir="./results_optuna",
        num_train_epochs=num_epochs,
        per_device_train_batch_size=batch_size_train,
        per_device_eval_batch_size=batch_size_eval,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        learning_rate=learning_rate,
        logging_steps=100,
        eval_strategy="epoch",
        save_strategy="no",
        report_to="none",
        seed=RANDOM_STATE,
        fp16=torch.cuda.is_available(),
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()

    return eval_results["eval_f1"]

# # 캐시 초기화
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()

# Optuna 실행 (기본 모델로 최적화)
tokenizer = AutoTokenizer.from_pretrained("../tapt-electra")  # Optuna용 토크나이저
study = optuna.create_study(direction="maximize")
study.optimize(objective_electra, n_trials=1)
best_params_electra = study.best_params
print(f"최적 하이퍼파라미터: {best_params_electra}")

# 최적 파라미터 적용
electra_BATCH_SIZE_TRAIN = best_params_electra["batch_size_train"]
electra_BATCH_SIZE_EVAL = best_params_electra["batch_size_eval"]
electra_LEARNING_RATE = best_params_electra["learning_rate"]
electra_NUM_EPOCHS = best_params_electra["num_epochs"]
electra_WARMUP_STEPS = best_params_electra["warmup_steps"]
electra_WEIGHT_DECAY = best_params_electra["weight_decay"]
electra_CHOSEN_MAX_LENGTH = best_params_electra["max_length"]
print("electra_BATCH_SIZE_TRAIN:", electra_BATCH_SIZE_TRAIN)
print("electra_BATCH_SIZE_EVAL:", electra_BATCH_SIZE_EVAL)
print("electra_LEARNING_RATE:", electra_LEARNING_RATE)
print("electra_NUM_EPOCHS:", electra_NUM_EPOCHS)
print("electra_WARMUP_STEPS:", electra_WARMUP_STEPS)
print("electra_WEIGHT_DECAY:", electra_WEIGHT_DECAY)
print("electra_CHOSEN_MAX_LENGTH:", electra_CHOSEN_MAX_LENGTH)

[I 2025-10-30 03:53:36,182] A new study created in memory with name: no-name-0eac2cce-ceb2-4091-ba53-99b7b534b223
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at ../tapt-electra and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.407900,0.417179,0.843876,0.839559
2,0.310200,0.384960,0.856606,0.854580


[I 2025-10-30 04:32:19,254] Trial 0 finished with value: 0.8545798550009707 and parameters: {'learning_rate': 9.420482412443099e-05, 'num_epochs': 2, 'batch_size_train': 64, 'batch_size_eval': 64, 'warmup_steps': 685, 'weight_decay': 0.04553144387239281, 'max_length': 256}. Best is trial 0 with value: 0.8545798550009707.


최적 하이퍼파라미터: {'learning_rate': 9.420482412443099e-05, 'num_epochs': 2, 'batch_size_train': 64, 'batch_size_eval': 64, 'warmup_steps': 685, 'weight_decay': 0.04553144387239281, 'max_length': 256}
electra_BATCH_SIZE_TRAIN: 64
electra_BATCH_SIZE_EVAL: 64
electra_LEARNING_RATE: 9.420482412443099e-05
electra_NUM_EPOCHS: 2
electra_WARMUP_STEPS: 685
electra_WEIGHT_DECAY: 0.04553144387239281
electra_CHOSEN_MAX_LENGTH: 256


# 완료

In [17]:
print("완료")

완료
